In [4]:
import pandas as pd
import bsa_utils

In [5]:
bsa_utils.show_available_datasets()

23362
25521
consolidated-pharmaceutical-list
contractor-details
dental-activity-delivered-by-newly-qualified-foundation-dentists
dispensing-practice-name-and-address
english-contractor-monthly-general-dental-activity
english-contractor-monthly-general-dental-and-orthodontic-contractual-dataset
english-contractor-monthly-orthodontic-activity
english-prescribing-data-epd
english-prescribing-dataset-epd-with-snomed-code
hormone-replacement-therapy-prescription-prepayment-certificate-hrt-ppc-volumes
hospital-prescribing-dispensed-in-the-community
national-uda-by-performer-for-financial-years-2016-2024
opendata-nhsbsa-net-dataset-foi-25455
open-data-portal-usage
pharmacy-contract-fp57-refunds
pharmacy-openings-and-closures
pharmacy-quality-scheme-data-2022-23
prescriber-details
prescription-cost-analysis-pca-annual-statistics
prescription-cost-analysis-pca-monthly-data
regional-uda-by-performer
secondary-care-medicines-data
secondary-care-medicines-data-indicative-price


In [6]:
#FIND NEW PRODUCTS
dataset_id = "english-prescribing-data-epd" # Dateset ID
date_from = "201801" # Can be "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "latest-1" # Can be "YYYYMM" or "latest" or "latest-1", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION "
    "{FROM_TABLE}"
)
results = bsa_utils.FetchData("english-prescribing-data-epd", sql, "latest-3", "latest", cache=False).results

INFO:root:Success for https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?resource_id=EPD_202401&sql=SELECT%20DISTINCT%20BNF_CODE%2C%20BNF_DESCRIPTION%20FROM%20%60EPD_202401%60
INFO:root:Success for https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?resource_id=EPD_202404&sql=SELECT%20DISTINCT%20BNF_CODE%2C%20BNF_DESCRIPTION%20FROM%20%60EPD_202404%60
INFO:root:Success for https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?resource_id=EPD_202403&sql=SELECT%20DISTINCT%20BNF_CODE%2C%20BNF_DESCRIPTION%20FROM%20%60EPD_202403%60
INFO:root:Success for https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?resource_id=EPD_202402&sql=SELECT%20DISTINCT%20BNF_CODE%2C%20BNF_DESCRIPTION%20FROM%20%60EPD_202402%60
INFO:root:Processing response data
INFO:root:Data processing complete


In [8]:
print (results)

<bound method FetchData.results of <bsa_utils.FetchData object at 0x7f1f64f15820>>


In [2]:
#FIND NEW PRODUCTS
dataset_id = "english-prescribing-data-epd" # Dateset ID
date_from = "201801" # Can be "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "latest-1" # Can be "YYYYMM" or "latest" or "latest-1", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION "
    "{FROM_TABLE}"
)

# Fetch data using BSA API
df_existing=bsa_utils.fetch_data(resource=dataset_id, date_from=date_from, date_to=date_to, sql=sql, cache=True)

FetchData("english-prescribing-data-epd", sql_query, "latest-3", "latest", cache=True)

# Save data to file
#df.to_csv("../data/data.csv", index=False)

date_from = "latest" # Can be "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "" # Can be "YYYYMM" or "latest" or "latest-1", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION, CHEMICAL_SUBSTANCE_BNF_DESCR "
    "{FROM_TABLE}"
)

# Fetch data using BSA API
df_latest=bsa_utils.fetch_data(resource=dataset_id, date_from=date_from, date_to=date_to, sql=sql)

# Save data to file
#df_latest.to_csv("../data/data.csv", index=False)

# Compare 2 dataframe - return rows with items which are only in new dataframe.
new_items=bsa_utils.show_new_items(df_existing, df_latest)

# Temporarily set the display option to show all rows for one print statement
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    display(new_items)

NameError: name 'bsa_utils' is not defined

In [4]:
#FIND NEW INHALERS
dataset_id = "english-prescribing-data-epd" # Dateset ID
date_from = "latest" # Can be either "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "latest" # Can be either "YYYYMM" or "latest", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION "
    "{FROM_TABLE}"
)

# Fetch data using BSA API
df_existing=bsa_utils.fetch_data(resource=dataset_id, date_from=date_from, date_to=date_to, sql=sql)

# Save data to file
#df.to_csv("../data/data.csv", index=False)

date_from = "latest" # Can be either "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "latest" # Can be either "YYYYMM" or "latest", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION, CHEMICAL_SUBSTANCE_BNF_DESCR "
    "{FROM_TABLE}"
)

# Fetch data using BSA API
df_latest=bsa_utils.fetch_data(resource=dataset_id, date_from=date_from, date_to=date_to, sql=sql)

# Save data to file
#df_latest.to_csv("../data/data.csv", index=False)

# Compare 2 dataframe - return rows with items which are only in new dataframe.
new_items=bsa_utils.show_new_items(df_existing, df_latest)

# Temporarily set the display option to show all rows for one print statement
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    display(new_items)

Initializing fetch_data function
Getting a list of available datasets based on date range
Generating list of API calls
Processing API calls with retries
API call successful for URL: https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?resource_id=EPD_202404&sql=SELECT%20DISTINCT%20BNF_CODE%2C%20BNF_DESCRIPTION%20FROM%20%60EPD_202404%60


NameError: name 'res' is not defined